In [6]:
import os
from azure.core.credentials import AzureKeyCredential
from langchain.schema import HumanMessage
from langchain.chat_models import AzureChatOpenAI
os.environ["AZURE_OPENAI_API_KEY"]  = 'b82effcf491e45a088b1cd578713311c'
os.environ["OPENAI_API_VERSION"]    = '2023-05-15'
os.environ["AZURE_OPENAI_ENDPOINT"] = 'https://usesharedaopenai001.openai.azure.com/'
os.environ["OPENAI_API_TYPE"]       = 'azure'
# Variables-------------------------------------------------------
index                               = "azure-cognitive-search-vector-demo"
azure_search_endpoint               = 'https://genai0.search.windows.net'
MODEL                               = "gtp35turbo-latest"
key                                 = 'lvhCA67EeE3JRyxyem5L0wGJSfOxscm2jft887ECdJAzSeDzoCNZ'
model                               = "text-embedding-ada-002"
credential                          = AzureKeyCredential(key)
COMPLETION_TOKENS                   = 1000
top_search_vector_k                 = 5

In [7]:
llm = AzureChatOpenAI(model_name=MODEL)

In [10]:
from langchain import PromptTemplate

In [31]:
# create the prompt, here we use multiple inputs
prompt = PromptTemplate(
    template=""""/
    You are a helpful Name Entity Recognition System. From the following text:    
    
    <<{text}>>
    
    Extract this target word class: '{word_class}'.
    Return the extracted word class with quotes. 
    For single word class of type 'date' return it in the format YYYY-MM-DD. 
    For intervals with a start date and end date return both dates in a list, for example: ["2021-07-01", "2022-07-01"]. 
    For word class 'company_name' return it in capital letters.
    """,
    input_variables=["word_class", "text"]
)

query = "What is the revenue of Pfizer from 2018-jul-01 to 2019-jul-01"
word_class = "dates"
#word_class = "company_name"

# format the prompt to add variable values
prompt_formatted_str: str = prompt.format(
    word_class = word_class,
    text = query,
    )

In [32]:
# make a prediction
prediction = llm.predict(prompt_formatted_str)

# print the prediction
print(prediction)

["2018-07-01", "2019-07-01"]
